## Install dependencies

Make sure to use GPU 
`Runtime` -> `Change Runtime Type`

In [ ]:
# Install dependancies
!pip install numpy pandas scikit-learn scipy tqdm
!pip install obspy pytorch-lightning lightning wandb
!pip install git+https://github.com/nikitadurasov/masksembles
!pip install git+https://github.com/crimeacs/PhaseHunter

In [ ]:
# Clone repo
!git clone https://github.com/crimeacs/PhaseHunter

In [ ]:
# Due to a bag Colab x ObsPy we need to restart runtime, simply continue after that cell
import os
os.kill(os.getpid(), 9)

## Load model

In [ ]:
import torch
from phasehunter.model import PhaseHunter
from phasehunter.dataloader import Augmentations, Waveforms_dataset

# Auto select device, note that PhaseHunter works best on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Linux
# device = torch.device("mps") # Mac

model = PhaseHunter.load_from_checkpoint('PhaseHunter/ckpts/30s_STEAD_decay-epoch=196.ckpt')
model = model.eval().float().to(device)

## Download some data

In [ ]:
import obspy
from obspy.clients.fdsn import Client as FDSN_Client

# Connect to the FDSN client (in this case USGS)
client = FDSN_Client("SCEDC")

# Get the Ridgecrest earthquake from USGS ComCat
t1 = obspy.UTCDateTime("2019-07-04T00:00:00")
t2 = obspy.UTCDateTime("2019-07-05T00:00:00")

cat = client.get_events(starttime=t1, endtime=t2, minmagnitude=5, minlatitude=35, maxlatitude=36, minlongitude=-118, maxlongitude=-117, includearrivals=True)
event = cat[0]

# Get origin time of the event
origin_time = event.origins[0].time

# Define start and end time for the waveform retrieval
starttime = origin_time + 5  # Some seconds after the event
endtime = starttime + 30  # Total 30 seconds of data

# Network and station information
net = "CI"
sta = "WBS"

# Get the three-component data
st = client.get_waveforms(network=net, station=sta, location="*", channel="BH?", starttime=starttime, endtime=endtime)

station_picks = []
# Fetch the P and S picks
for pick in event.picks:
    if pick.waveform_id.station_code == sta:
        station_picks.append(pick)

# Make sure that sampling rate is 100Hz
st.resample(100);
# Plot the data
st.plot();

## How to use PhaseHunter

In [ ]:
predictions = model.process_continuous_waveform(st)
predictions

## How to use PhaseHunter (advanced)

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Convert stream into torch tensor
input_data = np.vstack([x.data for x in st])

# Preprocess data
demean_data = Augmentations().demean(input_data)
normalized_data = Augmentations().normalize(demean_data)

# Make sure that segment is 30s
cropped_data = normalized_data[:,:3000]

# Convert to tensor
tensor_data = torch.tensor(cropped_data, dtype=torch.float32)

# Now, for uncertainty estimation we need to create a batched copy of the input
inference_sample = torch.stack([tensor_data]*128).to(device)

# Make predictions
predictions, embeddings = model(inference_sample)

# Post process predictions
p_pred = predictions[:,0].detach().cpu()
s_pred = predictions[:,1].detach().cpu()
embeddings = torch.mean(embeddings, axis=0).detach().cpu().numpy()

# Estimate uncertainty and get the most likely values 
p_dist, p_kde, p_val, p_uncert = model.get_likely_val(p_pred)
s_dist, s_kde, s_val, s_uncert = model.get_likely_val(s_pred)

# Plot results
fig, ax = plt.subplots(nrows=2, figsize=(10,5), sharex=True)

ax[0].plot(cropped_data[0], lw=0.7, color='black')
ax[0].axvline(p_val.item()*cropped_data.shape[-1], color='red', label='P pick')
ax[0].axvline(s_val.item()*cropped_data.shape[-1], color='blue', label='S pick')
ax[0].set_ylabel('Amplitude')

ax[1].plot(p_dist*cropped_data.shape[-1], p_kde(p_dist)*cropped_data.shape[-1], color='red')
ax[1].plot(s_dist*cropped_data.shape[-1], s_kde(s_dist)*cropped_data.shape[-1], color='blue')
ax[1].set_yticks([])
ax[1].set_ylabel('Pick uncertainties')

plt.subplots_adjust(hspace=0)